In [ ]:
import os
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
from astropy.time import Time
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import h5py
import pandas as pd
from sqlalchemy import func
import tqdm
from scipy.stats import beta

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold
from twoface.mass import get_m2_min, mf, period_at_surface, asini

In [ ]:
spt = Table.read('../data/EEM_dwarf_UBVIJHK_colors_Teff.txt', 
                 format='ascii.commented_header', header_start=21)
spt = spt[np.isfinite(spt['Msun'])]

In [ ]:
sb9 = QTable.read('../../twoface/data/sb9.fit')
sb9_orb = QTable.read('../../twoface/data/sb9_orbits.fit')
sb9 = join(sb9, sb9_orb, keys='Seq')
sb9_dwarf = sb9[np.array(['V' in s and 'IV' not in s and s[:3] in spt['SpT'] for s in sb9['Sp1']])]
sb9_dwarf = sb9_dwarf[sb9_dwarf['Grade'] >= 4]
# sb9_giant = sb9[np.array(['III' in s and 'IV' not in s for s in sb9['Sp1']])]

In [ ]:
sb9_dwarf['M1'] = [float(spt['Msun'][spt['SpT'] == x[:3]]) for x in sb9_dwarf['Sp1']]
sb9_dwarf['M2'] = [float(spt['Msun'][spt['SpT'] == x[:3]]) if x[:3] in spt['SpT'] else 0 for x in sb9_dwarf['Sp2']]
sb9_dwarf['R1'] = [float(spt['R_Rsun'][spt['SpT'] == x[:3]]) for x in sb9_dwarf['Sp1']]
sb9_dwarf['logg'] = np.log10((G * sb9_dwarf['M1']*u.Msun / (sb9_dwarf['R1']*u.Rsun)**2).to(u.cm/u.s**2).value)

In [ ]:
P_surf = period_at_surface(sb9_dwarf['M1']*u.Msun, sb9_dwarf['logg'], sb9_dwarf['e'],
                           M2=sb9_dwarf['M2']*u.Msun)
P = sb9_dwarf['Per']

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

P_ratio = (P/P_surf).decompose()
err_P_ratio = P_ratio * np.sqrt((sb9_dwarf['e_Per'] / sb9_dwarf['Per'])**2 + (0.05)**2)

ax.scatter(P_ratio, sb9_dwarf['e'], alpha=0.5, linewidth=0)
ax.errorbar(P_ratio, sb9_dwarf['e'], 
            xerr=err_P_ratio,
            yerr=sb9_dwarf['e_e'], 
            alpha=0.5, marker='', linestyle='none')
ax.set_xscale('log')
ax.set_xlim(5e-1, 1e3)
ax.set_ylim(-0.02, 1.02)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.scatter(P, sb9_dwarf['e'], alpha=0.5, linewidth=0)
ax.set_xscale('log')
ax.set_xlim(0.1, 1e4)